# Real-world images

Tento dataset se vám stáhnout nepodaří. Jedná se o obrázky ze soutěže Kaggle: https://www.kaggle.com/datasets/alessiocorrado99/animals10

Abyste tento dataset mohli také stáhnout, museli byste se na Kaggle zaregistrovat a stáhnout si jejich token pro programátorský přístup k datům.

## Stažení dat

In [ ]:
import json
import os
from google.colab import files
files.upload()

In [ ]:
!sudo mkdir -p ~/.kaggle
!sudo cp kaggle.json ~/.kaggle/
!sudo chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d alessiocorrado99/animals10
!unzip animals10.zip

In [ ]:
!ls

In [ ]:
parent_dir = 'raw-img'
italian = os.listdir(parent_dir)
italian

In [ ]:
from translate import translate
translate['ragno'] = 'spider'

for old_dir in italian:
  if os.path.exists(os.path.join(parent_dir, old_dir)):
    os.rename(os.path.join(parent_dir, old_dir), os.path.join(parent_dir, translate[old_dir]))

categories = os.listdir(parent_dir)
categories

## Příprava dat

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, TensorDataset, random_split
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [ ]:
dataset = ImageFolder(parent_dir)
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}

In [ ]:
dataset[0], dataset[1]

In [ ]:
# Náhodný obrázek a jeho kategorie
i = random.randint(0, len(dataset)-1)
img, cat = dataset[i]
plt.imshow(img)
plt.title(idx_to_class[cat])
plt.axis('off')
plt.show()

In [ ]:
# Data před posláním do neuronové sítě transformujeme
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize images to 64x64
    transforms.ToTensor(),  # Convert PIL image to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize tensors
])

In [ ]:
dataset = ImageFolder(parent_dir, transform=transform)
dataset[0], dataset[1]

In [ ]:
# Náhodný obrázek po zmenšení
i = random.randint(0, len(dataset)-1)
img, cat = dataset[i]
plt.imshow(img.permute(1, 2, 0) * 0.5 + 0.5)
plt.title(idx_to_class[cat])
plt.axis('off')
plt.show()

In [ ]:
rows, cols = 3, 6
fig, axes = plt.subplots(rows, cols, figsize=(10, 5))
for _, ax in enumerate(axes.flat):
  i = random.randint(0, len(dataset)-1)
  x, y = dataset[i]
  ax.imshow(x.permute(1, 2, 0) * 0.5 + 0.5)
  ax.set_title(idx_to_class[y])
  ax.axis('off')
plt.tight_layout()

In [ ]:
# Převeď celý dataset do velkého tensoru, ve kterém můžeme *rychle* přistupovat k částem
# Cca tři minuty - torch je nachystaný pro práci s *velkými datasety*, které
# se nevydají celé do paměti. V našem případě můžeme vše prostě načíst.
# V tomto případě zrychlíme přístup neuronové sítě k obrázkům přibližně stokrát.
all_images = torch.zeros(size=(len(dataset), 3, 64, 64))
all_labels = torch.zeros(size=(len(dataset), 1), dtype=torch.int64)

t0 = datetime.now()
print(f'Starting reading at {t0}')
for i in range(len(dataset)):
  all_images[i], all_labels[i] = dataset[i]
elapsed = datetime.now() - t0

all_labels = all_labels[:, 0].long()

print(f'Elapsed {elapsed}s, {1000 * elapsed / len(dataset)}s per 1k images.')

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tensor_dataset = TensorDataset(all_images.to(device), all_labels.to(device))

train_size = int(0.8 * len(tensor_dataset))
test_size = len(tensor_dataset) - train_size

# Divide the dataset by randomly selecting samples
train_dataset, test_dataset = random_split(tensor_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

## Konvoluční neuronová síť

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout(0.25)

        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout(0.25)

        self.fc1 = nn.Linear(256 * 8 * 8, 4096)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(4096, 4096)
        self.dropout5 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.dropout1(self.pool1(F.relu(self.conv2(x))))

        x = F.relu(self.conv3(x))
        x = self.dropout2(self.pool2(F.relu(self.conv4(x))))

        x = F.relu(self.conv5(x))
        x = self.dropout3(self.pool3(F.relu(self.conv6(x))))

        x = x.view(-1, 256 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout4(x)
        x = F.relu(self.fc2(x))
        x = self.dropout5(x)
        x = self.fc3(x)
        return x


In [ ]:
# Také potřebujeme GPU, jinak bude trénink velice pomalý.
# Rozumíte tomu, proč GPU / grafická karta pomáhá s trénováním neuronových sítí?
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
# Check if CUDA is available and print the GPU name
if torch.cuda.is_available():
    print("GPU is available")
    print("Name of the GPU: ", torch.cuda.get_device_name())
else:
    print("GPU is not available")

In [ ]:
def train_network(model, train_loader, test_loader, num_epochs=20):
  cross_entropy_loss = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.01)

  train_losses = []
  test_losses = []
  train_accuracies = []
  test_accuracies = []

  # Loss and accuracy before the training.
  model.eval()
  test_loss = 0.0
  test_correct = 0
  with torch.no_grad():
    for images, labels in test_loader:
      outputs = model(images)
      loss = cross_entropy_loss(outputs, labels)
      test_loss += loss.item() * images.size(0)
      _, predicted = torch.max(outputs.data, 1)
      test_correct += (predicted == labels).sum().item()

  test_loss = test_loss / len(test_loader.dataset)
  test_losses.append(test_loss)
  test_accuracy = test_correct / len(test_loader.dataset)
  test_accuracies.append(test_accuracy)

  train_losses.append(np.nan)
  train_accuracies.append(np.nan)

  t0 = datetime.now()
  print(f"Starting training at {t0} on device {next(model.parameters()).device}."
        f" Test loss {test_loss:.4f}, accuracy {100 * test_accuracy:.2g}%.")
  print()

  for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    model.train(True)
    for images, labels in train_loader:
      outputs = model(images)
      loss = cross_entropy_loss(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      train_loss += loss.item() * images.size(0)
      _, predicted = torch.max(outputs.data, 1)
      train_correct += (predicted == labels).sum().item()

    model.eval()
    test_loss = 0.0
    test_correct = 0
    with torch.no_grad():
      for images, labels in test_loader:
        outputs = model(images)
        loss = cross_entropy_loss(outputs, labels)
        test_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        test_correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    train_accuracy = train_correct / len(train_loader.dataset)
    train_accuracies.append(train_accuracy)

    test_loss = test_loss / len(test_loader.dataset)
    test_losses.append(test_loss)
    test_accuracy = test_correct / len(test_loader.dataset)
    test_accuracies.append(test_accuracy)

    print(f"Elapsed {datetime.now() - t0}s, epoch {epoch + 1} / {num_epochs}.",
          f"Training loss {train_loss:.4f}, accuracy {100 * train_accuracy:.1f}%.",
          f"Test loss {test_loss:.4f}, accuracy {100 * test_accuracy:.1f}%.")

  print()
  print(f"Finished in {datetime.now() - t0}s.")
  return train_losses, train_accuracies, test_losses, test_accuracies

In [ ]:
# Epocha trvá cca půlminuty na CPU...
model = ConvNet().to(device)
num_epochs = 50 if torch.cuda.is_available() else 5
train_loss, train_acc, test_loss, test_acc = train_network(
    model, train_loader, test_loader, num_epochs=num_epochs)

In [ ]:
plt.plot(train_loss, 'g', label='Train loss')
plt.plot(test_loss, 'r', label='Test loss')
plt.title('Loss')
plt.grid()
plt.legend()

In [ ]:
plt.plot(train_acc, 'g', label='Train accuracy')
plt.plot(test_acc, 'r', label='Test accuracy')
plt.title('Accuracy')
plt.grid()
plt.legend()

## Overfitting!

Co se stalo? Model je tak velký, že si dokázal "zapamatovat" celá data a přestal
se učit.

Právě toto je jednou z hlavních výzev umělé inteligence - jak zařídit, aby si síť pouze nezapamatovala data, ale naučila se skutečně něco podstatného.

**Jak si mohl model "zapamatovat" všechny obrázky?**

In [ ]:
model

In [ ]:
for p in model.parameters():
  print(p.shape, p.requires_grad)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Celkový počet parametrů modelu: {count_parameters(model):,d}')

In [ ]:
model = model.eval()
rows, cols = 5, 8
correct = 0
fig, axes = plt.subplots(rows, cols, figsize=(16, 8))
for i, ax in enumerate(axes.flat):
  x, y = test_dataset[random.randint(0, test_size)]
  y_pred = model(x.unsqueeze(0))
  _, y_guess = torch.max(y_pred.data, 1)
  y_guess = y_guess.item()
  correct += y_guess == y
  ax.imshow(x.cpu().permute(1, 2, 0) * 0.5 + 0.5)
  title = idx_to_class[y_guess] + ("" if y_guess == y else " - WRONG!")
  ax.set_title(title)
  ax.axis('off')
acc = correct / (rows * cols)
fig.suptitle(f'Test sample accuracy = {100 * acc:.3g}%')
fig.tight_layout()
model = model.train()

In [ ]:
model = model.eval()
rows, cols = 5, 8
correct = 0
fig, axes = plt.subplots(rows, cols, figsize=(16, 8))
for i, ax in enumerate(axes.flat):
  x, y = train_dataset[random.randint(0, train_size)]
  y_pred = model(x.unsqueeze(0))
  _, y_guess = torch.max(y_pred.data, 1)
  y_guess = y_guess.item()
  correct += y_guess == y
  ax.imshow(x.cpu().permute(1, 2, 0) * 0.5 + 0.5)
  title = idx_to_class[y_guess] + ("" if y_guess == y else " - WRONG!")
  ax.set_title(title)
  ax.axis('off')
acc = correct / (rows * cols)
fig.suptitle(f'Train sample accuracy = {100 * acc:.3g}%')
plt.tight_layout()
model = model.train()

Model si "zapamatoval" trénovací data, ale nedokáže dostatečně dobře zobecňovat na testovací data. Dataset obsahuje necelých 30 tisíc obrázků, takový dataset
je velice malý. Rovněž použitá neuronová síť je relativně jednoduchá v porovnání s velkými modely trénované Google, Microsoft apod (ačkoli měřeno počtem parametrů je velká).

In [ ]:
import torchvision.models as models
alexet = models.alexnet(num_classes=10)

In [ ]:
# Vítěz soutěže ImageNet 2012 - AlexNet, top-5 error rate 15.3%
# Tato síť je poměrně "jednoduchá" a na našem dataset také nefunguje lépe než
# naše stávající konvoluční síť.
alexnet

In [ ]:
print(f'Celkový počet parametrů modelu: {count_parameters(alexnet):,d}')

In [ ]:
# ResNet, vítěz ImageNet 2015 od Microsoft
resnet = models.resnet152(num_classes=10)
resnet

In [ ]:
print(f'Celkový počet parametrů modelu: {count_parameters(googlenet):,d}')

Ani tento model nefunguje na našem datasetu lépe než náš. Dataset je příliš malý.

Velkou inženýrskou výzvou při vytváření umělé inteligence je jak efektivně trénovat na velkých datech a mnoha GPU zároveň.